Mini Project
# Custom Question answering



Goal of this question answering is to find the answer to a question given a question and a accompanying context.Predicted answer will be a text from the context or a empty string(which indicates that question can not be answered from the context).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!pip install simpletransformers
import simpletransformers

In [3]:
import json
with open(r"/content/drive/MyDrive/files/train.json") as read_file:
  train = json.load(read_file)


In [4]:
train

[{'context': 'Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Who is the author of the Mistborn series?',
    'answers': [{'text': 'Brandon Sanderson', 'answer_start': 71}]}]},
 {'context': 'The first series, published between 2006 and 2008, consists of The Final Empire,The Well of Ascension, and The Hero of Ages.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'When was the series published?',
    'answers': [{'text': 'between 2006 and 2008', 'answer_start': 28}]},
   {'id': '00003',
    'is_impossible': False,
    'question': 'What are the three books in the series?',
    'answers': [{'text': 'The Final Empire, The Well of Ascension, and The Hero of Ages',
      'answer_start': 63}]},
   {'id': '00004',
    'is_impossible': True,
    'question': 'Who is the main character in the series?',
    'answers': []}]}]

In [5]:
import json
with open(r"/content/drive/MyDrive/files/test.json") as read_file:
  test = json.load(read_file)


In [6]:
test

[{'context': 'The series primarily takes place in a region called the Final Empire on a world called Scadrial, where the sun and sky are red, vegetation is brown, and the ground is constantly being covered under black volcanic ashfalls.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'Where does the series take place?',
    'answers': [{'text': 'region called the Final Empire', 'answer_start': 38},
     {'text': 'world called Scadrial', 'answer_start': 74}]}]},
 {'context': '"Mistings" have only one of the many Allomantic powers, while "Mistborns" have all the powers.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'How many powers does a Misting possess?',
    'answers': [{'text': 'one', 'answer_start': 21}]},
   {'id': '00003',
    'is_impossible': True,
    'question': 'What are Allomantic powers?',
    'answers': []}]}]

**QuestionAnsweringArgs** seems to be an argument class or configuration settings for a question-answering model.

QuestionAnsweringModel is likely a class that allows you to build and use a **question-answering model**. This class may provide methods and attributes for training, evaluating, and using a model for question-answering tasks.

In [10]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel,QuestionAnsweringArgs

I can use any Transformer model given below.

In [11]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

The **QuestionAnswringModel** class will takes two arguments one is **model_type** and second one is **model_name**.

In [12]:
# configure the model
model_args = QuestionAnsweringArgs()

In [13]:
# i'm going to setup some  arguments for my model
model_args.train_batch_size = 16
model_args.n_best_size = 3
model_args.num_train_epochs = 20
model_args.evaluate_during_training = True

Here **n_best_size** **= 3** means i want three different types of outputs

In [14]:
# i'm not going to use above model_args . Because there are more number of arguments we want to setup.

In [15]:
# Now i'm going to install a library known as Wandb for visualizing the model
!pip install wandb

In [21]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs":20,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [22]:
# initialize the model it will download the pre-trained model
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Train the model
model.train_model(train, eval_data=test)


convert squad examples to features: 100%|██████████| 4/4 [00:00<00:00, 158.33it/s]

add example index and unique id: 100%|██████████| 4/4 [00:00<00:00, 16256.99it/s]


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

correct,▁▁▁▁▁
eval_loss,█▆▄▂▁
global_step,▁▃▅▆█
incorrect,▁▁▁▁▁
similar,▁▁▁▁▁
train_loss,██▅▂▁
correct,0
eval_loss,-0.53369
global_step,5
incorrect,2
similar,1


Running Epoch 0 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Running Epoch 18 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(20,
 {'global_step': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20],
  'correct': [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'similar': [2, 2, 3, 3, 2, 2, 2, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
  'incorrect': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'train_loss': [4.9114580154418945,
   4.96875,
   4.705078125,
   4.1770830154418945,
   3.7014975547790527,
   3.1259765625,
   2.5885417461395264,
   1.9326171875,
   1.6826171875,
   1.4596354961395264,
   1.2884114980697632,
   1.0084636211395264,
   0.8784586787223816,
   0.9574788808822632,
   0.83074951171875,
   0.7045491337776184,
   0.7135009765625,
   0.6324869394302368,
   0.5926309823989868,
   0.7314859628677368],
  'eval_loss': [0.02215576171875,
   -0.0028362274169921875,
   -0.057586669921875,
   -0.11773681640625,
   -0.1734619140625,
   -0.23095703125,
   -0.29345703125,
   -0.358642578125,
  

You can see the graphs of our **bert** **model** for 5 epochs here: https://drive.google.com/file/d/1P6R4jziIsw3HCHcp409e60GuEH5yhFK4/view?usp=sharing
And for 20 epochs : https://drive.google.com/file/d/1ctz5cDVG-LwKi35CZesmKPnlEB2YCLNE/view?usp=sharing


 you can change the model into **roberta**, **distilbert**, **distilroberta** etc as your wish.

In [24]:
# Let's evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
result

{'correct': 1, 'similar': 2, 'incorrect': 0, 'eval_loss': -0.9287109375}

we can see that the **correct** prediction is **one** and **incorrect** prediction is **0**. And there are **2** **similar** predictions

In [27]:
# Let's make prediction with this example
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]


In [28]:
answers, probabilities = model.predict(to_predict)

print(answers)


convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 94.53it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4457.28it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['', 'great power']}]


we can see that it **predicted** as **correct**.